In [2]:
%load_ext autoreload
%autoreload 2
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine as cosine_distance
import numpy as np
from itertools import combinations
from tqdm import tqdm
import sys
sys.path.append('../src')
from models import produce_effect_size, calculate_association_metric_for_target_word, get_matrices_from_term_lists

In [2]:
wv_from_bin = KeyedVectors.load_word2vec_format("../data/external/GoogleNews-vectors-negative300.bin", binary=True)  # C bin format

In [10]:
wv_from_bin['science']

array([ 0.0090332 ,  0.12158203,  0.3984375 ,  0.21386719, -0.01672363,
        0.06396484,  0.3125    , -0.23535156, -0.10009766, -0.09326172,
        0.20507812, -0.03222656, -0.21191406,  0.00958252, -0.27734375,
        0.09619141, -0.1875    , -0.04052734,  0.2265625 , -0.23632812,
       -0.04858398,  0.03295898, -0.08056641,  0.12255859,  0.05859375,
       -0.09130859, -0.02685547,  0.02587891, -0.11376953, -0.21972656,
       -0.0378418 , -0.15722656, -0.05981445, -0.01635742,  0.11425781,
       -0.15234375, -0.01086426,  0.29296875,  0.22265625, -0.00245667,
       -0.16308594, -0.11669922, -0.20800781,  0.31835938, -0.08740234,
        0.13378906,  0.15429688,  0.11816406, -0.07470703,  0.09326172,
       -0.32421875,  0.01239014, -0.07226562, -0.10986328, -0.07177734,
        0.16113281,  0.05541992, -0.3984375 , -0.04980469, -0.25195312,
       -0.05249023, -0.05786133,  0.16699219, -0.26953125, -0.04394531,
       -0.14257812,  0.03662109,  0.34765625, -0.10888672, -0.12

In [7]:
# WEAT 6
X_terms = ['John', 'Paul','Mike','Kevin','Steve','Greg','Jeff','Bill']
Y_terms = ['Amy','Joan','Lisa','Sarah','Diana','Kate','Ann','Donna']
A_terms = ['executive','management','professional','corporation',
               'salary','office','business','career']
B_terms = ['home','parents','children','family',
               'cousins','marriage','wedding','relatives']

In [15]:
# WEAT 7 using Google News word2vec model
X_terms = ['math','algebra','geometry','calculus',
             'equations','computation','numbers','addition']
Y_terms = ['poetry','art','dance','literature',
             'novel','symphony','drama','sculpture']
A_terms = ['male','man','boy','brother',
              'he','him','his','son']
B_terms = ['female','woman','girl','sister',
               'she','her','hers','daughter']

In [19]:
# Weat 8
A_terms = ['brother','father','uncle','grandfather',
          'son','he','his','him']
B_terms = ['sister','mother','aunt','grandmother',
          'daughter','she','hers','her']
X_terms = ['science','technology','physics','chemistry',
          'Einstein','NASA','experiment','astronomy']
Y_terms = ['poetry','art','Shakespeare','dance',
          'literature','novel','symphony','drama']

In [55]:
wv_test_object = {'x_word1':[.1,.2,.3],
            'x_word2':[.4,.5,.6],
            'y_word1':[.2,.4,.6],
            'y_word2':[.1,.3,.5],
            'a_word1':[.5,.7,.9],
            'a_word2':[.4,.6,.8],
            'b_word1':[.3,.6,.9],
            'b_word2':[.4,.7,.9]}

In [58]:
X_terms = ['x_word1', 'x_word2']
Y_terms = ['y_word1', 'y_word2']
A_terms = ['a_word1', 'a_word2']
B_terms = ['b_word1', 'b_word2']
[X_mtx, Y_mtx, A_mtx, B_mtx] = get_matrices_from_term_lists(wv_test_object, X_terms, Y_terms, A_terms, B_terms)

In [61]:
produce_effect_size(wv_test_object, X_terms, Y_terms, A_terms, B_terms)

1.1899801283181513

# Test Statistic

In [66]:
x1_vec = wv_test_object['x_word1']
s1 = calculate_association_metric_for_target_word(x1_vec, A_mtx, B_mtx)

In [68]:
x2_vec = wv_test_object['x_word2']
s2 = calculate_association_metric_for_target_word(x2_vec, A_mtx, B_mtx)

In [74]:
y1_vec = wv_test_object['y_word1']
sy1 = calculate_association_metric_for_target_word(y1_vec, A_mtx, B_mtx)
y2_vec = wv_test_object['y_word2']
sy2 = calculate_association_metric_for_target_word(y2_vec, A_mtx, B_mtx)

In [75]:
print(s1, s2, s1+s2 - (sy1+sy2))

-0.007639423266564194 0.013706608780193585 0.03175706038191817


In [77]:
def produce_test_statistic(X_mtx, Y_mtx, A_mtx, B_mtx):
    x_associations = np.apply_along_axis(lambda x_vec: calculate_association_metric_for_target_word(x_vec, A_mtx, B_mtx), 1, X_mtx)
    y_associations = np.apply_along_axis(lambda y_vec: calculate_association_metric_for_target_word(y_vec, A_mtx, B_mtx), 1, Y_mtx)
    return np.sum(x_associations)
produce_test_statistic(X_mtx, Y_mtx, A_mtx, B_mtx)

0.006067185513629392

In [85]:
x = ['A','B','C','D','E','F']
for seq in combinations(set(x), len(x)//2):
    print(seq, set(x).difference(seq))

('F', 'C', 'D') {'A', 'E', 'B'}
('F', 'C', 'A') {'E', 'B', 'D'}
('F', 'C', 'E') {'A', 'B', 'D'}
('F', 'C', 'B') {'A', 'E', 'D'}
('F', 'D', 'A') {'C', 'E', 'B'}
('F', 'D', 'E') {'A', 'C', 'B'}
('F', 'D', 'B') {'A', 'C', 'E'}
('F', 'A', 'E') {'C', 'B', 'D'}
('F', 'A', 'B') {'C', 'E', 'D'}
('F', 'E', 'B') {'A', 'C', 'D'}
('C', 'D', 'A') {'F', 'E', 'B'}
('C', 'D', 'E') {'F', 'A', 'B'}
('C', 'D', 'B') {'F', 'A', 'E'}
('C', 'A', 'E') {'F', 'B', 'D'}
('C', 'A', 'B') {'F', 'E', 'D'}
('C', 'E', 'B') {'F', 'A', 'D'}
('D', 'A', 'E') {'F', 'C', 'B'}
('D', 'A', 'B') {'F', 'C', 'E'}
('D', 'E', 'B') {'F', 'A', 'C'}
('A', 'E', 'B') {'F', 'C', 'D'}


In [ ]:
x_union_y = set(X_terms).union(set_Y_terms)
for 

In [4]:
a_union_b = {'a','b','c','d'}
already_seen = []
for seq in combinations(a_union_b, 2):
    complement = a_union_b.difference(seq)
    already_seen.append(complement)
    if set(seq) in already_seen:
        continue
    print(seq, complement)

('d', 'a') {'c', 'b'}
('d', 'b') {'a', 'c'}
('d', 'c') {'a', 'b'}


In [6]:
print(len(a_union_b))

4


In [23]:
def get_complements(x_union_y):
    '''Generator function that yields pairs of equal-size disjoint subsets
    of x_union_y.
    x_union_y should a set type.'''
    already_seen = []
    for seq in combinations(x_union_y, len(x_union_y)//2):
        complement = x_union_y.difference(seq)
        already_seen.append(complement)
        if set(seq) in already_seen:
            continue
        yield (seq, complement)

In [21]:
x_union_y = {'a','b','c','d'}
for tup in get_complements(x_union_y):
    print(tup)

(('d', 'a'), {'c', 'b'})
(('d', 'b'), {'a', 'c'})
(('d', 'c'), {'a', 'b'})
